In [204]:
import io
import requests
import pandas as pd
import numpy as np
import networkx as nx
import nxpd
import matplotlib.pyplot as plt
import matplotlib
import pyspark
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql import dataframe
import graphframes

from src.data_reader import DataReader
from src.to_gexf import to_gexf

In [182]:
reader = DataReader('data/s2-corpus-00')

In [187]:
reader.write() if not written else None
written = True

In [189]:
edges = spark.createDataFrame(reader.read_edges(50e3), schema=StructType(
[
    StructField(name='src', dataType=StringType()),
    StructField(name='dst', dataType=StringType())
]
)).withColumn('id', F.monotonically_increasing_id())


nodes = spark.createDataFrame(reader.read_nodes(10e3), schema=StructType(
[
    StructField(name='id', dataType=StringType()),
    StructField(name='year', dataType=StringType()),
    StructField(name='authors', dataType=StringType()),
    StructField(name='entities', dataType=StringType())
]
))


In [190]:
g = graphframes.GraphFrame(e=edges, v=nodes)
to_gexf(g, 'data/graph.gexf')

In [197]:
page_rank = g.pageRank(maxIter=5, resetProbability=0.9)

GraphFrame(v:[id: string, year: string ... 3 more fields], e:[src: string, dst: string ... 2 more fields])

In [203]:
page_rank.edges.show()

+--------------------+--------------------+-----------+------+
|                 src|                 dst|         id|weight|
+--------------------+--------------------+-----------+------+
|15ed7a529360639e5...|38bb76891ab91b37e...|       7063|   1.0|
|d6b5d08c93118db6a...|3c7420db0fd835840...|       8965|   0.5|
|d6b5d08c93118db6a...|3c7420db0fd835840...|       9769|   0.5|
|d6b5d08c93118db6a...|3c7420db0fd835840...|       8965|   0.5|
|d6b5d08c93118db6a...|3c7420db0fd835840...|       9769|   0.5|
|0d8f5e3988670deef...|aaa09419253fbd5b6...|       8580|   0.5|
|0d8f5e3988670deef...|aaa09419253fbd5b6...| 8589939659|   0.5|
|0d8f5e3988670deef...|aaa09419253fbd5b6...|       8580|   0.5|
|0d8f5e3988670deef...|aaa09419253fbd5b6...| 8589939659|   0.5|
|381a7328fac4a99c4...|32bb822b32a485933...| 8589935568|   0.5|
|381a7328fac4a99c4...|32bb822b32a485933...|17179872269|   0.5|
|381a7328fac4a99c4...|32bb822b32a485933...| 8589935568|   0.5|
|381a7328fac4a99c4...|32bb822b32a485933...|17179872269|

In [221]:
e = g.edges.toPandas().iloc[:, [0,1]]
e.columns = ['source', 'target']
G = nx.DiGraph(e)

In [223]:
nxpd.draw(G)

KeyboardInterrupt: 